In [ ]:
## GPT를 활용해서 Few shot inference를 수행합니다.
## 하나의 data 생성에 30초정도 걸립니다... korquad의 unique한 context 9000개를 활용해서 
## 문제를 제작할 경우 약 16시간정도의 시간이 걸립니다.
import openai
from datasets import load_dataset
import pandas as pd
import numpy as np
from ast import literal_eval

client = openai.OpenAI(
    api_key='YOUR API KEY'
)


In [16]:
import unicodedata

def normalize_unicode(text):
    # 유니코드 문자열을 ASCII로 정규화
    return unicodedata.normalize("NFKD", text)

def generate(row):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """당신은 문제의 해설을 작성하는 수능 출제자입니다.
                               paragraph와 question을 읽고 answer에 대한 근거를 작성하세요.
                               형식 외에 다른 출력을 내보내지 마세요.
                               반드시 근거는 200자 이내로 작성하세요.
                               반드시 근거 : 내용 형식을 사용하세요.
                               """,
            },
            {
                "role": "user",
                "content": f""" paragraph : {row['paragraph']}
                                question : {row['question']}
                                answer : {row['answer']}
                                근거 : [model output]
                            """
            },
        ],
    )
    response = response.choices[0].message.content.strip()
    row = {
        "id": f"yongari_generate_{int(str(abs(hash(row['paragraph'])))[:5])}",
        "paragraph": row['paragraph'],
        "problems": row['problems'],
        "klue" : response +' # '+str(row['answer']),
        "question_plus": np.nan,

    }
    return row


In [17]:
from tqdm import tqdm
from ast import literal_eval
data = pd.read_csv('datas/train.csv')

data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['question'] = data['dic'].apply(lambda x: x['question'])
data['answer'] = data['dic'].apply(lambda x: x['answer'])
generated = []
for idx, row in tqdm(data.iterrows()):
    generated.append(generate(row))

generated = pd.DataFrame(generated)

2031it [1:07:29,  1.99s/it]


In [24]:
generated.to_csv('train+klue.csv', index = False)